# Lista 7

## Zadanie 2

Wykorzystując architekturę sieci neuronowych RetinaNet Focal Loss for Dense Object Detection zaprogramuj wykrywanie obiektów.

In [ ]:
import keras

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

def grab_camera_snapshot(camera_id=0, fallback_filename='zdjecie.jpg'):
    camera = cv2.VideoCapture(camera_id)
    try:
        for i in range(15):
            snapshot_ok, image = camera.read()
        if not snapshot_ok:
            print("Ups: could not access camera")
            if fallback_filename:
                image = read_image_bgr(fallback_filename)
    finally:
        camera.release()
    return image

gpu = 0
setup_gpu(gpu)

# wybieramy model
model_path = os.path.join('.', '', 'resnet50_coco_best_v2.1.0.h5')
model = models.load_model(model_path, backbone_name='resnet50')

labels_to_names = {
    0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
    5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
    10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
    14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
    20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
    25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
    30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite',
    34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard',
    37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass',
    41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl',
    46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli',
    51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake',
    56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed',
    60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse',
    65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave',
    69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book',
    74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear',
    78: 'hair drier', 79: 'toothbrush'}

# ładujemy obrazek, moze byc algo konkretny albo z kamery
#image = read_image_bgr('003.jpg')
image = grab_camera_snapshot()

# tworzenie kopii obrazka po której będziemy malować i którą wyświetlimy
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# przygotowanie obrazka
image = preprocess_image(image)
image, scale = resize_image(image)

# przepuszczenie obrazka przez sieć
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# przeskalowanie naszych pudełek ktore będziemy rysować na obrazku
boxes /= scale

# jeżeli score byl > 50 to wtedy rysujemy na obrazku
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores są posortowane więc jak trafimy na mniejszy niż 0.5 to mozemy dalej nie sprawdzać
    # chociaż z ciekawości ustawilem na mniejszy
    # wiecej rysuje, ale i więcej sie myli, czego się spodziwałem
    if score < 0.5:
        break
    color = label_color(label)
    b = box.astype(int)
    draw_box(draw, b, color=color)
    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)

plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()

## Zadanie 3

Zapoznaj się z różnymi modelami dostępnymi w Kerasie https://keras.io/applications/ oraz wykładem 12. Wybierz architekturę MobileNet lub Squeeze-and-Excitation network. Przeczytaj publikację i samodzielnie zaimplementuj ją w Kerasie. Postaraj się uzyskać dobrą wydajność na małych zbiorach danych np. CIFAR-10.

Próbowałem zastosować tą sieć do obrazków 32x32, momo że prawdopodobnie lepiej by było gdybym przeskalował obrazki do tych wielkości które są podane w dokumentacji(np 224x224). Próbowałem zastosować to co jest robione w implementacji keras-a czyli usunąć tzw "top" (w tym wypadku go zedytować) w jakiś sposób żeby ilość wejść nie zanikała całkowicie. Mimo to widzę, że moja siecie nie uzyskuje dobrej wydajności ale nie zdążyłem tego naprawić przed końcem terminu.

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time



def myMobileNet(alpha):
    num_classes = 10
    inputs = tf.keras.layers.Input(shape=(32, 32, 3))

    x = Conv(inputs, filters=32, kernel_size=3, strides=2, alpha=alpha)
    x = ConvDw(x, filters=32, strides=1, alpha=alpha)
    x = Conv(x, filters=64, kernel_size=1, alpha=alpha)
    x = ConvDw(x, filters=64, strides=2, alpha=alpha)
    x = Conv(x, filters=128, kernel_size=1, alpha=alpha)
    x = ConvDw(x, filters=128, strides=1, alpha=alpha)
    x = Conv(x, filters=128, kernel_size=1, alpha=alpha)
    x = ConvDw(x, filters=128, strides=2, alpha=alpha)
    x = Conv(x, filters=256, kernel_size=1, alpha=alpha)
    x = ConvDw(x, filters=256, strides=1, alpha=alpha)
    x = Conv(x, filters=256, kernel_size=1, alpha=alpha)
    x = ConvDw(x, filters=256, strides=2, alpha=alpha)
    x = Conv(x, filters=512, kernel_size=1, alpha=alpha)
    for i in range( 5 ):
        x = ConvDw(x, filters=512 , strides=1 , alpha=alpha)
        x = Conv(x, filters=512 , kernel_size=1 , alpha=alpha)
    x = ConvDw(x, filters=512, strides=2, alpha=alpha)
    x = Conv(x, filters=1024, kernel_size=1, alpha=alpha)
    x = tf.keras.layers.AveragePooling2D(pool_size=(1, 1))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(num_classes)(x)
    outputs = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.models.Model(inputs, outputs)
    return model

def ConvDw( x , filters , strides , alpha=1.0 ):
    x = tf.keras.layers.DepthwiseConv2D( kernel_size=3 , padding='same' )( x )
    x = tf.keras.layers.BatchNormalization(momentum=0.999)( x )
    x = tf.keras.layers.Activation( 'relu' )( x )
    x = tf.keras.layers.Conv2D( np.floor( filters * alpha ) , kernel_size=( 1 , 1 ) , strides=strides , use_bias=False , padding='same' )( x )
    x = tf.keras.layers.BatchNormalization(momentum=0.999)(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def Conv( x , filters , kernel_size , strides=1 , alpha=1.0 ):
    x = tf.keras.layers.Conv2D( np.floor( filters * alpha ) , kernel_size=kernel_size , strides=strides , use_bias=False , padding='same' )( x )
    x = tf.keras.layers.BatchNormalization( momentum=0.999 )(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x



(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = np.array( x_train ) / 255
y_train = np.array( y_train )
x_test = np.array( x_test ) / 255
y_test = np.array( y_test )
model = myMobileNet(1.0)
model.summary()
n_epochs = 15

model.compile(
    optimizer=optimizers.SGD(lr=0.1),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    x=x_train, y=y_train,
    epochs=n_epochs,
)
model.evaluate(x=x_test, y=y_test, verbose=1)